In [1]:
import gymnasium as gym
import pogema
from pogema import GridConfig
from stable_baselines3 import DQN, A2C
from stable_baselines3.common.evaluation import evaluate_policy

%load_ext autoreload
%autoreload 2
%matplotlib inline

GRID_LEN = 18

def custom_step(self, action: list):
        assert len(action) == self.grid_config.num_agents
        rewards = []

        terminated = []

        self.move_agents(action)
        self.update_was_on_goal()

        for agent_idx in range(self.grid_config.num_agents):

            c_x, c_y = self.grid.positions_xy[agent_idx]
            f_x, f_y = self.grid.finishes_xy[agent_idx]

            #d = math.sqrt((c_x - f_x) ** 2 + (c_y - f_y) ** 2)    
            #reward = 1 - (d / (math.sqrt(2) * GRID_LEN))
            reward = 1 - ( (abs(c_x - f_x) + abs(c_y - f_y)) / (2 * GRID_LEN) )
            #print(f"[CURR] {c_x}, {c_y} [FINISH] {f_x}, {f_y} [DIST] {d} [REWARD] {reward}")


            on_goal = self.grid.on_goal(agent_idx)
            if on_goal and self.grid.is_active[agent_idx]:
                print("FINISH", reward)
                rewards.append(reward)
            else:
                rewards.append(reward)
            terminated.append(on_goal)

        for agent_idx in range(self.grid_config.num_agents):
            if self.grid.on_goal(agent_idx):
                self.grid.hide_agent(agent_idx)
                self.grid.is_active[agent_idx] = False

        infos = self._get_infos()

        observations = self._obs()
        truncated = [False] * self.grid_config.num_agents
        return observations, rewards, terminated, truncated, infos

pogema.envs.Pogema.step = custom_step

grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    max_episode_steps=30
)

env = gym.make("Pogema-v0",grid_config=grid_config)

a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


#### Optuna Integration

In [13]:
""" Optuna example that optimizes the hyperparameters of
a reinforcement learning agent using A2C implementation from Stable-Baselines3
on a Gymnasium environment.

This is a simplified version of what can be found in https://github.com/DLR-RM/rl-baselines3-zoo.

You can run this example as follows:
    $ python sb3_simple.py

"""
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(3.0e5)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3


ENV_ID = "Pogema-v0"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "verbose": 1,
    "env": ENV_ID
}

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for A2C hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    gae_lambda = 1.0 - trial.suggest_float("gae_lambda", 0.001, 0.2, log=True)
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    # Display true values.
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("gae_lambda_", gae_lambda)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]} if net_arch == "tiny" else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "gae_lambda": gae_lambda,
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
            "ortho_init": ortho_init,
        },
    }


class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_a2c_params(trial))
    # Create the RL model.
    model = A2C(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))


[I 2023-11-05 15:16:13,512] A new study created in memory with name: no-name-ffdac970-5856-4ef4-a08a-ef78e840bab1
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(
/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary an

Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH

[I 2023-11-05 15:17:09,540] Trial 0 finished with value: 26.388889000000002 and parameters: {'gamma': 0.000887920374516703, 'max_grad_norm': 0.46651810122895704, 'gae_lambda': 0.09838458639329409, 'exponent_n_steps': 7, 'lr': 1.0130517726531976e-05, 'ent_coef': 0.002077946979820953, 'ortho_init': True, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 0 with value: 26.388889000000002.


Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 60.3     |
|    ep_rew_mean        | 51.1     |
| time/                 |          |
|    fps                | 5157     |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 12800    |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | 0.998    |
|    learning_rate      | 0.201    |
|    n_updates          | 99       |
|    policy_loss        | -0       |
|    value_loss         | 6.45     |
------------------------------------
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINI

...

Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 52       |
|    ep_rew_mean        | 44.6     |
| time/                 |          |
|    fps                | 4698     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 3200     |
| train/                |          |
|    entropy_loss       | -1.6     |
|    explained_variance | -0.134   |
|    learning_rate      | 0.000168 |
|    n_updates          | 99       |
|    policy_loss        | 19.1     |
|    value_loss         | 169      |
------------------------------------
FINISH 1.0
FINISH 1.0
FINI

[I 2023-11-05 15:26:11,493] Trial 9 pruned. 


Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 50.8     |
|    ep_rew_mean        | 43.2     |
| time/                 |          |
|    fps                | 4675     |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 6400     |
| train/                |          |
|    entropy_loss       | -1.61    |
|    explained_variance | -0

[I 2023-11-05 15:27:15,006] Trial 10 finished with value: 27.222222 and parameters: {'gamma': 0.09370444970800293, 'max_grad_norm': 0.3125073620325973, 'gae_lambda': 0.0021946320722501483, 'exponent_n_steps': 6, 'lr': 0.00018547055825582557, 'ent_coef': 3.4019421163688734e-06, 'ortho_init': True, 'net_arch': 'small', 'activation_fn': 'tanh'}. Best is trial 7 with value: 27.500000333333332.


Number of finished trials:  11
Best trial:
  Value:  27.500000333333332
  Params: 
    gamma: 0.011011927944382008
    max_grad_norm: 0.308177919886694
    gae_lambda: 0.002034246679473824
    exponent_n_steps: 9
    lr: 0.00020557686968677058
    ent_coef: 5.341462640708193e-06
    ortho_init: False
    net_arch: tiny
    activation_fn: relu
  User attrs:
    gamma_: 0.988988072055618
    gae_lambda_: 0.9979657533205262
    n_steps: 512


### Train agent with best hyper parameters

In [3]:
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
'''
# gave 49% for 1.25e5 steps
a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1,
    gamma=0.00018803899618193213,
    max_grad_norm=1.2862481809583681,
    gae_lambda=0.005001539671087738,
    learning_rate=0.0001865002271714901,
    ent_coef=2.4476585337430064e-06,
    n_steps=64,
    policy_kwargs={"net_arch": [{"pi": [64], "vf": [64]}], "activation_fn": nn.Tanh, "ortho_init": False}
)
'''

#3.0e5 steps
a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1,
    gamma=0.011011927944382008,
    max_grad_norm=0.308177919886694,
    gae_lambda=0.002034246679473824,
    learning_rate=0.00020557686968677058,
    ent_coef=5.341462640708193e-06,
    n_steps=512,
    policy_kwargs={"net_arch": [{"pi": [64], "vf": [64]}], "activation_fn": nn.ReLU, "ortho_init": False}
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [5]:
# Train agent and save it
a2c_model.learn(int(3.0e5))
a2c_model.save("saved/a2c_baseline")

FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0
FINISH 1.0

#### Testing 

In [6]:

a2c_model = A2C.load("saved/a2c_baseline")

env.reset()

mean_reward, std_reward = evaluate_policy(a2c_model, env, deterministic=True, n_eval_episodes=20)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

FINISH 1.0
FINISH 1.0
FINISH 1.0
mean_reward:23.02 +/- 9.09


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [7]:
# RANDOM SEED
from IPython.display import SVG, display
from pogema.animation import AnimationMonitor, AnimationConfig

env = AnimationMonitor(env)

def evaluate_success_rate(model, env, num_episodes=100):
    success_count = 0
    step_array = []
    for i in range(num_episodes):
        print(f'---{i}---')
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 100
        steps_taken = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done)
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                step_array.append(steps_taken)
                env.save_animation(f"render{i}.svg", AnimationConfig(egocentric_idx=0))
                break

    success_rate = success_count / num_episodes
    return success_rate, step_array

success_rate,step_array = evaluate_success_rate(a2c_model, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")

---0---
3 100 0 False
2 99 0 False
2 98 0 False
2 97 0 False
1 96 0 False
3 95 0 False
0 94 0 False
0 93 0 False
2 92 0 False
4 91 0 False
4 90 0 False
2 89 0 False
1 88 0 False
1 87 0 False
2 86 0 False
0 85 0 False
2 84 0 False
4 83 0 False
2 82 0 False
3 81 0 False
3 80 0 False
3 79 0 False
2 78 0 False
4 77 0 False
0 76 0 False
1 75 0 False
0 74 0 False
0 73 0 False
3 72 0 False
3 71 0 False
1 70 0 False
2 69 0 False
4 68 0 False
2 67 0 False
4 66 0 False
3 65 0 False
3 64 0 False
3 63 0 False
0 62 0 False
3 61 0 False
3 60 0 False
3 59 0 False
3 58 0 False
3 57 0 False
1 56 0 False
3 55 0 False
3 54 0 False
4 53 0 False
2 52 0 False
2 51 0 False
0 50 0 False
3 49 0 False
1 48 0 False
0 47 0 False
2 46 0 False
0 45 0 False
2 44 0 False
2 43 0 False
2 42 0 False
2 41 0 False
3 40 0 False
3 39 0 False
1 38 0 False
3 37 0 False
1 36 0 False
2 35 0 False
0 34 0 False
2 33 0 False
2 32 0 False
2 31 0 False
2 30 0 False
4 29 0 False
3 28 0 False
3 27 0 False
3 26 0 False
4 25 0 False
3 2

In [8]:
# we got 49% success rate for 1.25e5 steps
# we got 35% success rate for 3.0e5 steps 
# better to use 1.25e5 steps